In [ ]:
import os
import time
import json
%reload_ext e2emlstorlets.tools.ipython

In [ ]:
os.environ['OS_AUTH_VERSION'] = '3'
os.environ['OS_AUTH_URL'] = 'http://127.0.0.1:5000/v3'
os.environ['OS_USERNAME'] = 'tester'
os.environ['OS_PASSWORD'] = 'testing'
os.environ['OS_USER_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_NAME'] = 'test'

In [ ]:
%%storletapp extract_face.ExtractFace
import cv2
import numpy as np

def detect(im):
    mat=cv2.imdecode(im, cv2.IMREAD_GRAYSCALE)
    cascade = cv2.CascadeClassifier("/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_alt2.xml")
    rects = cascade.detectMultiScale(mat)

    if len(rects) == 0:
        return [], mat
    rects[:, 2:] += rects[:, :2]
    rect = rects[0]
    return mat, rect

def crop(img, rect):
    h = rect[3]-rect[1]
    w = rect[2]-rect[0]
    x = rect[0]
    y = rect[1]
    hm = int(0.1 * h)
    if y < hm:
        h = h + (hm - y)
        hm = y
    return img[y-hm:y+h, x:x+w]

class ExtractFace(object):
    def __init__(self, logger):
        self.logger = logger

    def __call__(self, in_files, out_files, params):
        metadata = in_files[0].get_metadata()
        out_files[0].set_metadata(metadata)

        # Read the image
        img_str = ''
        while True:
            buf = in_files[0].read(1024)
            if not buf:
                break
            img_str += buf
        img_nparr = np.fromstring(img_str, np.uint8)
        
        # Detect face
        mat, rect = detect(img_nparr)

        # Crop the face and decrease resolution
        face = crop(mat, rect)
        small_face = cv2.resize(face, (50,55))
        
        # Write result
        retval, small_face_buf = cv2.imencode('.jpg', small_face)
        out_files[0].write(small_face_buf)
        
        in_files[0].close()
        out_files[0].close()
        self.logger.debug('Done\n')

In [ ]:
# Iterate over all pictures, and extract faces
start_time = time.time()
%list_container -i train -o obj_list
for obj in obj_list:
    input_path=os.path.join('/train',obj)
    output_path=os.path.join('/extracted',obj)
    %copy \
        --storlet extract_face.py \
        --input path:$input_path \
        --output path:$output_path \
        -o result
extract_faces_time = time.time() - start_time
print('Done extracting faces in %d seconds' % round(extract_faces_time))

In [ ]:
# Show before and after face extraction
%show_image --input path:/train/bibi1.jpeg
%show_image --input path:/extracted/bibi1.jpeg

In [ ]:
# Train the model with all faces
start_time = time.time()
input_path = os.path.join('path:extracted/', obj_list[0])
output_path = 'path:/trained/model'
extra = ','.join([ '/extracted/%s' % obj_name for obj_name in obj_list[1:]])
%copy \
    --storlet train_model.py \
    --input $input_path \
    --output $output_path \
    --extra $extra \
    -o result
train_model_time = time.time() - start_time
print('Model training done in %d seconds' % round(train_model_time))

In [ ]:
%play_video -c test -v trump_mov.avi

In [ ]:
# Add face recognition tag
start_time = time.time()
input_path = 'path:/test/bibi_mov.avi'
output_path = 'path:/video/tagged_bibi_mov.avi'
extra_resources = '/trained/model'
%copy \
    --storlet video_recognize_face.py \
    --input $input_path \
    --output $output_path \
    --extra $extra_resources \
    -o result
recognize_faces_time = time.time() - start_time
print('Recognize face done in %d seconds' % round(recognize_faces_time))

In [ ]:
%play_video -c video -v tagged_bibi_mov.avi

In [ ]:
# now lets see how this performs when working against S3
from e2emlstorlets.s3 import extract_faces, \
    train_model, swap_faces, recognize_faces
start_time = time.time()
extract_faces.extract_and_upload_all()
extract_faces_s3_time = time.time() - start_time
print('Done extracting faces in %d seconds' % round(extract_faces_s3_time))

start_time = time.time()
train_model.train_and_upload_model()
train_model_s3_time = time.time() - start_time
print('Model training done in %d seconds' % round(train_model_s3_time))

start_time = time.time()
recognize_faces.get_tag_and_upload("bibi_mov.avi")
tag_faces_s3_time = time.time() - start_time
print('Recognizing faces done in %d seconds' % round(tag_faces_s3_time))

In [ ]:
from e2emlstorlets.tools import plot_times
plot_times.show_plot(extract_faces_time,
                     train_model_time,
                     recognize_faces_time,
                     extract_faces_s3_time,
                     train_model_s3_time,
                     tag_faces_s3_time)